In [32]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import time
df = pd.read_csv('distance_matrix.csv')
distance_matrix = np.round(df.values)
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = np.round(distance_matrix).astype(int)
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
         len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    t1 = time.time()
    solution = routing.SolveWithParameters(search_parameters)
    t2 = time.time()
    print(t2-t1)
    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)
    routes = get_routes(solution, routing, manager)
    return routes

if __name__ == "__main__":
    tour = main()

0.03492546081542969
Objective: 15832 miles
Route for vehicle 0:
 0 -> 1 -> 2 -> 5 -> 6 -> 28 -> 29 -> 31 -> 30 -> 25 -> 26 -> 23 -> 22 -> 33 -> 32 -> 20 -> 19 -> 21 -> 18 -> 15 -> 13 -> 14 -> 12 -> 17 -> 16 -> 24 -> 27 -> 11 -> 10 -> 7 -> 8 -> 9 -> 4 -> 3 -> 0



In [33]:
routine = pd.DataFrame(columns=["city","lat", "lon"])
df_start = pd.read_csv('cityinfo.csv')

for i in range(34):
    routine.loc[i] = df_start.loc[tour[0][i]]

import folium

# 创建一个中心位于北京的地图
m = folium.Map(location=[39.9042, 116.4074], zoom_start=5)

# 假设我们有一个包含城市坐标的数据集
cities_data = routine

# 添加城市标记
for index, row in cities_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['city'],
        icon=folium.Icon(color='red')
    ).add_to(m)

# 连接城市
temp = cities_data[['lat', 'lon']].values.tolist()
locations = np.vstack((temp, temp[0]))
folium.PolyLine(locations, color="blue", weight=2.5, opacity=1).add_to(m)

m.save('routine_ortools.html')